In [1]:
from common_preamble import *
from scipy import stats

searching for scad in: /Users/caseyschneider-mizell/anaconda3/envs/jupyter_space/bin:/Users/caseyschneider-mizell/.gem/ruby/2.7.0/bin:/usr/local/opt/ruby/bin:/Users/drpoo/anaconda3/bin:/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Applications/VMware Fusion.app/Contents/Public:/Library/TeX/texbin:/opt/X11/bin:/Applications/OpenSCAD.app/Contents/MacOS


Loading data from /Users/caseyschneider-mizell/Work/allen_work/data_analysis/ais_analysis_pipeline/data/in/data_v185_ais_bounds_v3_is_chandelier_v5.h5


Making new plot directory...
Minimum AIS length: 37187.05055806807
	Computing aggregate AIS information


In [2]:
data_dir = f'{base_dir}/data'

In [3]:
mm = trimesh_io.MeshMeta(disk_cache_path=mesh_dir,
                         cache_size=0, cv_path=mesh_cv_path)

ais_meshes = []
for oid in complete_ais_ids:
    ais_file = ais_mesh_dir + '/{}_ais.h5'.format(oid)
    if os.path.exists(ais_file):
        ais_mesh = mm.mesh(filename=ais_file)
        ais_meshes.append(ais_mesh)
    else:
        ais_meshes.append(None)
        print('{} AIS not found!'.format(oid))


In [4]:
import cloudvolume as cv
svs = cv.CloudVolume(mesh_cv_path, mip=5, bounded=False, use_https=True)

In [5]:
import cloudvolume as cv
from meshparty import skeletonize
def segmentation_mesh_zeros(mesh, dmax_vec, cv_path, mip=5, cap_y=True, use_sk=True, base_mip=0):
    svs = cv.CloudVolume(cv_path, mip=mip, bounded=False, use_https=True)
    print('\tComputing grid coordinates...')
    xmin, ymin, zmin = ((mesh.bounds[0] - max(dmax_vec)) / svs.resolution).astype(int)
    xmax, ymax, zmax = ((mesh.bounds[1] + max(dmax_vec)) / svs.resolution).astype(int)
    if cap_y:
        ymin = (mesh.bounds[0][1] / svs.resolution[1]).astype(int)
        ymax = (mesh.bounds[1][1] / svs.resolution[1]).astype(int)

    X, Y, Z = np.meshgrid(np.arange(xmin, xmax), np.arange(ymin,ymax), np.arange(zmin, zmax), indexing='ij')

    xyzs = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T * svs.resolution
    if use_sk:
        sk = skeletonize.skeletonize_mesh(mesh)
        vert_obj = sk
    else:
        vert_obj = mesh
        
    try:
        dsp, _ = vert_obj.pykdtree.query(xyzs, distance_upper_bound=max(dmax_vec))
    except:
        dsp, _ = vert_obj.kdtree.query(xyzs, distance_upper_bound=max(dmax_vec))

    print('\tDownloading segmentation...')

    xslice = slice(xmin, xmax)
    yslice = slice(ymin, ymax)
    zslice = slice(zmin, zmax)
    
    sv_dat = np.squeeze(svs[xslice, yslice, zslice])
    
    print('\tComputing proximate grid volume...')
    outside_seg = (sv_dat==0).astype(int).ravel()
    frac_out = []
    for dmax in dmax_vec:
        valid=dsp<dmax
        vox_outside = np.sum(outside_seg[valid])
        vox_total = np.sum(valid)
        frac_out.append(vox_outside / vox_total)
    return frac_out

In [8]:
old_version = 183
fraction_zeros_old_df = pd.read_hdf(data_dir + '/mask_fraction_data_v{}.hdf'.format(old_version),
                                    'mask_fraction_df')

In [9]:
dmax =  [5000, 7500, 10000, 15000]
mip=5
cap_y=True
f_all = {}
for oid, ais_mesh in tqdm.tqdm(zip(complete_ais_ids, ais_meshes), total=len(ais_meshes)):
    if np.any(fraction_zeros_old_df['root_id']==oid):
        f = fraction_zeros_old_df[fraction_zeros_old_df['root_id']==oid].values[0,1:]
    else:
        f = segmentation_mesh_zeros(ais_mesh, dmax, mesh_cv_path, mip, cap_y)
    f_all[oid] = f

    
d_dict = {k:{} for k in dmax}
for k, x in f_all.items():
    for ii, d in enumerate(dmax):
        d_dict[d][k] = float(x[ii])

        
df_cols = {'d_{}'.format(d): d_dict[d] for d in dmax}
mask_fraction_df = pd.DataFrame(df_cols).reset_index().rename(columns={'index':'root_id'})
mask_fraction_df.to_hdf(data_dir + '/mask_fraction_data_v{}.hdf'.format(data_version),
                        'mask_fraction_df')


  0%|          | 0/153 [00:00<?, ?it/s]

	Computing grid coordinates...




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 12.39it/s]




Downloading:   0%|          | 0/284 [00:00<?, ?it/s]

Downloading:   8%|▊         | 22/284 [00:03<00:38,  6.83it/s]

Downloading:   8%|▊         | 24/284 [00:03<00:37,  6.94it/s]

Downloading:   9%|▉         | 25/284 [00:04<01:06,  3.89it/s]

Downloading:  10%|▉         | 27/284 [00:04<01:07,  3.82it/s]

Downloading:  11%|█         | 31/284 [00:05<00:52,  4.81it/s]

Downloading:  12%|█▏        | 33/284 [00:05<00:46,  5.39it/s]

Downloading:  12%|█▏        | 34/284 [00:05<00:41,  5.96it/s]

Downloading:  12%|█▏        | 35/284 [00:05<00:39,  6.26it/s]

Downloading:  13%|█▎        | 36/284 [00:06<01:15,  3.30it/s]

Downloading:  13%|█▎        | 37/284 [00:06<01:23,  2.95it/s]

Downloading:  13%|█▎        | 38/284 [00:06<01:17,  3.16it/s]

Downloading:  14%|█▎        | 39/284 [00:06<01:02,  3.89it/s]

Downloading:  14%|█▍        | 40/284 [00:07<01:10,  3.48it/s]

Downloading:  14%|█▍        | 41/284 [00:07<01:07,  3.62it/s]

Downloading:  15%|█▍        | 42/284 [00:07<00:56,  4.32it/s]


	Computing proximate grid volume...


...

	Computing grid coordinates...




  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 8816/8816 [00:00<00:00, 359575.86it/s]


100%|██████████| 1/1 [00:00<00:00,  8.18it/s]




Downloading:   0%|          | 0/71 [00:00<?, ?it/s]

Downloading:  30%|██▉       | 21/71 [00:04<00:10,  4.83it/s]

Downloading:  31%|███       | 22/71 [00:04<00:12,  4.05it/s]

Downloading:  32%|███▏      | 23/71 [00:05<00:13,  3.63it/s]

Downloading:  34%|███▍      | 24/71 [00:05<00:12,  3.88it/s]

Downloading:  38%|███▊      | 27/71 [00:05<00:09,  4.85it/s]

Downloading:  41%|████      | 29/71 [00:05<00:07,  5.76it/s]

Downloading:  44%|████▎     | 31/71 [00:05<00:05,  6.83it/s]

Downloading:  46%|████▋     | 33/71 [00:06<00:04,  8.11it/s]

Downloading:  51%|█████     | 36/71 [00:06<00:03,  9.02it/s]

Downloading:  55%|█████▍    | 39/71 [00:06<00:03,  9.70it/s]

Downloading:  58%|█████▊    | 41/71 [00:08<00:13,  2.24it/s]

Downloading:  59%|█████▉    | 42/71 [00:09<00:11,  2.62it/s]

Downloading:  61%|██████    | 43/71 [00:09<00:09,  3.00it/s]

Downloading:  62%|██████▏   | 44/71 [00:09<00:08,  3.21it/s]

Downloading:  63%|██████▎   | 45/71 [00:09<00:06,  3.90it/s]

Downloading:  65%

	Computing proximate grid volume...



100%|██████████| 153/153 [1:02:35<00:00, 24.54s/it]


In [10]:
1

1